In [ ]:
__author__ = "imflash217"
__copyright__ = "FlashAI Labs, 2019"

In [ ]:
# magic commands
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# libraries
import os
import numpy as np
import pandas as pd
import matplotlib 
import scipy
import torch
from fastai.vision import *
from fastai.metrics import error_rate


In [ ]:
# setting paths
x_path = untar_data(URLs.PETS)
print(x_path)
print(x_path.ls())                               #support function by fastai library to list items in a path

x_path_annotations = x_path/'annotations'
x_path_images = x_path/'images'

In [ ]:
# exploring the data
x_fnames = get_image_files(x_path_images)
print(len(x_fnames))
print(type(x_fnames))
x_fnames[0:5]

In [ ]:
# creating the databunch for training
np.random.seed(217)                      # setting random seed for numpy
x_pattern = r'/([^/]+)_\d+.jpg$'         # regex pattern
x_batch_size = 6                           # batch size for loading data for training

# creating databunch
x_data = ImageDataBunch.from_name_re(x_path_images, x_fnames, x_pattern, ds_tfms=get_transforms(), size=224, bs=x_batch_size)\
         .normalize(imagenet_stats)

In [ ]:
x_data.show_batch(rows=2, figsize=(7,6))


In [ ]:
print(x_data.classes)
print(type(x_data.classes))
print(len(x_data.classes), x_data.c)

In [ ]:
# Transfer Learning from pretrained architecture
x_learn = create_cnn(data=x_data, arch=models.resnet34, metrics=error_rate)
print(x_learn.model)

In [ ]:
# Training the network
x_learn.fit_one_cycle(cyc_len=2)

In [ ]:
# saving the trained model
x_learn.save('stage1')

In [ ]:
x_interp = ClassificationInterpretation.from_learner(x_learn)
x_losses, x_idxs = x_interp.top_losses()
print(len(x_data.valid_ds)==len(x_losses)==len(x_idxs))
x_interp.plot_top_losses(k=9, figsize=(15,15))

In [ ]:
x_interp.plot_confusion_matrix(figsize=(10,10), dpi=90)

In [ ]:
x_interp.most_confused(min_val=3)

In [ ]:
x_learn.unfreeze()                    # unfreezing our model to train it a bit more
x_learn.fit_one_cycle(cyc_len=2)      # training the unfrozen model for another 2 epochs

In [ ]:
x_learn.load('stage1');

In [ ]:
x_learn.lr_find()

In [ ]:
x_learn.recorder.plot()

In [ ]:
x_learn.unfreeze()
x_learn.fit_one_cycle(cyc_len=2, max_lr=slice(1e-6,1e-4))

In [ ]:
x_data2 = ImageDataBunch.from_name_re(path=x_path_images, fnames=x_fnames, pat=x_pattern, ds_tfms=get_transforms(), size=299,bs=x_batch_size//2)\
            .normalize(imagenet_stats)
x_learn2 = create_cnn(data=x_data2, arch=models.resnet50, metrics=error_rate)
x_learn2.lr_find()
x_learn2.recorder.plot()

In [ ]:
x_learn2.fit_one_cycle(cyc_len=10)

In [ ]:
x_learn2.save('stage1-resnet50')
x_learn2.unfreeze()
x_learn2.fit_one_cycle(cyc_len=5, max_lr=slice(1e-5, 1e-2))

In [ ]:
x_path3 = untar_data(URLs.MNIST_SAMPLE);
print(x_path3)
x_transforms = get_transforms(do_flip=False)
x_data3 = ImageDataBunch.from_folder(path=x_path3, ds_tfms=x_transforms, size=26)

In [ ]:
x_data3.show_batch(rows=3, figsize=(3,3))
print(x_data3.classes)

In [ ]:
x_learn3 = create_cnn(data=x_data3, arch=models.resnet18, metrics=accuracy)
x_learn3.fit_one_cycle(cyc_len=2)

In [ ]:
x_dataframe = pd.read_csv(x_path3/'labels.csv')
x_dataframe.head()

In [ ]:
x_data4 = ImageDataBunch.from_df(path=x_path3, df=x_dataframe, ds_tfms=x_transforms, size=24)
print(x_data4.classes)

In [ ]:
x_filename_paths = [x_path3/name for name in x_dataframe['name']]
print(x_filename_paths[0:2])

In [ ]:
x_pattern4 = r"/(\d)/\d+\.png$"              # regex pattern
x_data4 = ImageDataBunch.from_name_re(path=x_path3, fnames=x_filename_paths, pat=x_pattern4, ds_tfms=x_transforms, size=24)
print(x_data4.classes)

In [ ]:
help(x_data.c)

In [ ]:
ImageDataBunch??